In [1]:
# install libraries
!pip install pandas matplotlib nltk webvtt-py

In [2]:
# Download NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab') # Added punkt_tab download

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# Imports
import re
import webvtt
import pandas as pd
from webvtt import WebVTT
from datetime import timedelta
from collections import Counter

from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [4]:
# Upload your files
from google.colab import files
uploaded = files.upload()

Saving all_comments.txt to all_comments.txt
Saving captions.en.vtt to captions.en.vtt


In [5]:
# Check Exiting Files in Directory
!ls

all_comments.txt  captions.en.vtt  sample_data


In [6]:
# Comment Parser
def structure_comments_from_txt(filepath='all_comments.txt'):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    comments_data = []

    # Each line looks like:
    # "1. @username [2025-10-19T22:03:01Z] 0 Likes → This was good!"
    pattern = re.compile(
        r'^\s*\d+\.\s*@([\w\-\_]+)\s*\[([0-9T:\-Z]+)\]\s*(\d+)\s*Likes?\s*→\s*(.*)$'
    )

    for line in lines:
        line = line.strip()
        match = pattern.match(line)
        if match:
            username = match.group(1)
            timestamp = match.group(2)
            likes = int(match.group(3))
            comment_text = match.group(4).strip()

            # Add length-based features
            char_length = len(comment_text)
            word_count = len(comment_text.split())
            avg_word_length = round(char_length / word_count, 2) if word_count else 0

            comments_data.append({
                'username': username,
                'timestamp_text': timestamp,
                'likes': likes,
                'comment_text': comment_text,
                'char_length': char_length,
                'word_count': word_count,
                'avg_word_length': avg_word_length
            })

    return pd.DataFrame(comments_data)

comments_df = structure_comments_from_txt()
comments_df.head()

,username,timestamp_text,likes,comment_text,char_length,word_count,avg_word_length
0,heidierwin3546,2025-10-19T22:03:01Z,0,"This was good, but probably needs to be update...",107,18,5.94
1,magdalenaojeda3315,2025-10-18T08:30:23Z,0,This is amazing!,16,3,5.33
2,anadikulkarni5039,2025-10-14T11:22:03Z,0,Ngl this was just a giant Twilio advertisement.,47,8,5.88
3,F8ferrari888,2025-10-11T00:32:33Z,0,Are you Mr Beasts&#39; father? lol. Why do you...,115,23,5.00
4,FlexTransportationServicesLLC,2025-10-09T21:31:10Z,0,I would like to reach out to you. I have been ...,252,47,5.36


In [7]:
# Comment Parser
def structure_comments_from_txt(filepath='all_comments.txt'):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    comments_data = []

    # Each line looks like:
    # "1. @username [2025-10-19T22:03:01Z] 0 Likes → This was good!"
    pattern = re.compile(
        r'^\s*\d+\.\s*@([\w\-\_]+)\s*\[([0-9T:\-Z]+)\]\s*(\d+)\s*Likes?\s*→\s*(.*)$'
    )

    for line in lines:
        line = line.strip()
        match = pattern.match(line)
        if match:
            username = match.group(1)
            timestamp = match.group(2)
            likes = int(match.group(3))
            comment_text = match.group(4).strip()

            # Add length-based features
            char_length = len(comment_text)
            word_count = len(comment_text.split())
            avg_word_length = round(char_length / word_count, 2) if word_count else 0

            comments_data.append({
                'username': username,
                'timestamp_text': timestamp,
                'likes': likes,
                'comment_text': comment_text,
                'char_length': char_length,
                'word_count': word_count,
                'avg_word_length': avg_word_length
            })

    return pd.DataFrame(comments_data)

comments_df = structure_comments_from_txt()
comments_df.head()

,username,timestamp_text,likes,comment_text,char_length,word_count,avg_word_length
0,heidierwin3546,2025-10-19T22:03:01Z,0,"This was good, but probably needs to be update...",107,18,5.94
1,magdalenaojeda3315,2025-10-18T08:30:23Z,0,This is amazing!,16,3,5.33
2,anadikulkarni5039,2025-10-14T11:22:03Z,0,Ngl this was just a giant Twilio advertisement.,47,8,5.88
3,F8ferrari888,2025-10-11T00:32:33Z,0,Are you Mr Beasts&#39; father? lol. Why do you...,115,23,5.00
4,FlexTransportationServicesLLC,2025-10-09T21:31:10Z,0,I would like to reach out to you. I have been ...,252,47,5.36


In [8]:
# Cleaning pipeline
stop_words = set(stopwords.words('english'))
lemmatizer, stemmer = WordNetLemmatizer(), PorterStemmer()

def normalize_text(text: str) -> str:
    if text is None: return ""
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'[^a-z\s]', ' ', text)      # change to [^a-z0-9\s] to keep digits
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_text(text: str):
    if not text: return []
    return word_tokenize(text)

def remove_stopwords(tokens):
    return [w for w in tokens if w not in stop_words and len(w) > 2]

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(w) for w in tokens]

def stem_tokens(tokens):
    return [stemmer.stem(w) for w in tokens]

def clean_text_pipeline(text: str, use_lemmatization: bool = True):
    x = normalize_text(text)
    toks = tokenize_text(x)
    toks = remove_stopwords(toks)
    return lemmatize_tokens(toks) if use_lemmatization else stem_tokens(toks)

# Comments parser
def structure_comments_from_txt(filepath='all_comments.txt'):
    pat = re.compile(
        r'^\s*\d+\.\s*@([\w\-]+)\s*\[([0-9T:\-Z]+)\]\s*([\d,]+)\s*Likes?\s*→\s*(.+)$'
    )
    rows = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for raw in f:
            m = pat.match(raw.strip())
            if not m:
                continue
            username  = m.group(1)
            timestamp = m.group(2)
            likes = int(m.group(3).replace(',', ''))
            comment = m.group(4).strip()
            rows.append({'username': username,
                         'timestamp': timestamp,
                         'likes': likes,
                         'comment_text': comment})
    return pd.DataFrame(rows, columns=['username','timestamp','likes','comment_text'])

# Captions parser
_SENT_SPLIT = re.compile(r'(?<=[.!?])\s+')

def _c(s: str) -> str:
    # minimal clean: strip, collapse spaces, drop stray quotes
    s = ' '.join(s.strip().split())
    return s.strip('"\''"“”‘’")

def _deoverlap(prev_line: str, curr_line: str, k: int = 6) -> str:
    """
    Remove up to k-word overlap between the end of prev_line and start of curr_line.
    This kills the rolling-window repetition common in auto-captions.
    """
    a = prev_line.split()
    b = curr_line.split()
    for n in range(min(k, len(a), len(b)), 0, -1):
        if a[-n:] == b[:n]:
            return ' '.join(b[n:])
    return curr_line

def structure_captions_from_vtt(filepath='captions.en.vtt'):
    # read all cues (assumes webvtt works; no extra error handling)
    cues = [_c(cue.text) for cue in WebVTT().read(filepath) if cue.text.strip()]
    if not cues:
        return pd.DataFrame({'caption_sentence': []})

    # merge cues while removing small overlaps
    merged = [cues[0]]
    for t in cues[1:]:
        extra = _deoverlap(merged[-1], t, k=6)
        if extra:
            merged[-1] = merged[-1] + ' ' + extra

    # split into sentences
    big_text = ' '.join(merged)
    sentences = [s.strip() for s in _SENT_SPLIT.split(big_text) if s.strip()]

    return pd.DataFrame({'caption_sentence': sentences})

# Preview
comments_df = structure_comments_from_txt('all_comments.txt')
comments_df['cleaned_tokens'] = comments_df['comment_text'].apply(clean_text_pipeline)
print("Comments head:")
print(comments_df.head())

captions_df = structure_captions_from_vtt('captions.en.vtt')
captions_df['cleaned_tokens'] = captions_df['caption_sentence'].apply(clean_text_pipeline)
print("\nCaptions head:")
print(captions_df.head())

Comments head:
                        username             timestamp  likes  \
0                 heidierwin3546  2025-10-19T22:03:01Z      0   
1             magdalenaojeda3315  2025-10-18T08:30:23Z      0   
2              anadikulkarni5039  2025-10-14T11:22:03Z      0   
3                   F8ferrari888  2025-10-11T00:32:33Z      0   
4  FlexTransportationServicesLLC  2025-10-09T21:31:10Z      0   

                                        comment_text  \
0  This was good, but probably needs to be update...   
1                                   This is amazing!   
2    Ngl this was just a giant Twilio advertisement.   
3  Are you Mr Beasts&#39; father? lol. Why do you...   
4  I would like to reach out to you. I have been ...   

                                      cleaned_tokens  
0  [good, probably, need, updated, example, used,...  
1                                          [amazing]  
2                [ngl, giant, twilio, advertisement]  
3  [beast, father, lol, look, like, c

In [9]:
# RESULTS & DISCUSSION

STOP = set(stopwords.words('english'))

def _count_sw(text):
    if pd.isna(text): return 0
    toks = word_tokenize(str(text).lower().strip())
    return sum(1 for t in toks if t in STOP)

def _top_tokens(df, col='cleaned_tokens', k=10):
    bag = Counter()
    for toks in df.get(col, []):
        if isinstance(toks, list): bag.update(toks)
    return bag.most_common(k)

def _issues(df, col):
    empty = (df[col].fillna("").str.strip() == "").sum()
    nonascii = df[col].fillna("").map(lambda s: any(ord(c)>127 for c in str(s))).sum()
    symbols = df[col].fillna("").map(
        lambda s: (len(str(s)) - sum(ch.isalnum() or ch.isspace() or ch in ".,!?'" for ch in str(s))) >= 3
    ).sum()
    return empty, nonascii, symbols

print("\n=== RESULTS & DISCUSSION ===")

# 1) Sizes
print(f"- Rows → comments: {comments_df.shape[0]} | captions: {captions_df.shape[0]}")

# 2) Before→After examples (2 each)
def _norm(s): return str(s).lower().strip().replace("\n"," ")
print("\nExamples — Comments (before → tokens)")
for _, r in comments_df[['comment_text','cleaned_tokens']].head(2).iterrows():
    print("•", _norm(r['comment_text'])[:120])
    print("  →", r['cleaned_tokens'][:12])

print("\nExamples — Captions (before → tokens)")
for _, r in captions_df[['caption_sentence','cleaned_tokens']].head(2).iterrows():
    print("•", _norm(r['caption_sentence'])[:120])
    print("  →", r['cleaned_tokens'][:12])

# 3) Exact stopword totals (report-ready single line)
com_sw = comments_df['comment_text'].apply(_count_sw).sum() if 'comment_text' in comments_df else 0
cap_sw = captions_df['caption_sentence'].apply(_count_sw).sum() if 'caption_sentence' in captions_df else 0
print(f"\nStopwords removed (exact count on originals): total={int(com_sw+cap_sw)} "
      f"(comments={int(com_sw)}, captions={int(cap_sw)})")

# 4) Top tokens (10)
print("\nTop tokens — Comments (10)")
print(_top_tokens(comments_df, 'cleaned_tokens', 10))
print("Top tokens — Captions (10)")
print(_top_tokens(captions_df, 'cleaned_tokens', 10))

# 5) Input issues (counts only)
if 'comment_text' in comments_df:
    e,n,s = _issues(comments_df,'comment_text')
    print(f"\nData issues — Comments: empty={e}, non_ascii={n}, symbols={s}")
if 'caption_sentence' in captions_df:
    e,n,s = _issues(captions_df,'caption_sentence')
    print(f"Data issues — Captions: empty={e}, non_ascii={n}, symbols={s}")

# 6) Engagement (only if likes available)
if 'likes' in comments_df:
    comments_df['tok_len']  = comments_df['cleaned_tokens'].map(lambda x: len(x) if isinstance(x,list) else 0)
    comments_df['char_len'] = comments_df['comment_text'].astype(str).str.len()
    comments_df['word_len'] = comments_df['comment_text'].astype(str).str.split().map(len)
    print("\nEngagement — correlations with likes")
    for c in ['word_len','tok_len','char_len']:
        print(f"  corr(likes, {c}) = {comments_df['likes'].corr(comments_df[c]):.3f}")
    print("\nTop 3 most-liked comments (user | likes | snippet)")
    for _, r in comments_df.sort_values('likes', ascending=False).head(3).iterrows():
        user = r['username'] if 'username' in comments_df else 'user'
        snippet = str(r['comment_text']).replace("\n"," ")[:80]
        print(f"  • {user} | {r['likes']} | {snippet}")


=== RESULTS & DISCUSSION ===
- Rows → comments: 963 | captions: 62

Examples — Comments (before → tokens)
• this was good, but probably needs to be updated. both examples used (spotify &amp; twilo) are now outdated.
  → ['good', 'probably', 'need', 'updated', 'example', 'used', 'spotify', 'amp', 'twilo', 'outdated']
• this is amazing!
  → ['amazing']

Examples — Captions (before → tokens)
• this course is an excellent introduction to apis for beginners this is an updated to apis for beginners this is an updat
  → ['course', 'excellent', 'introduction', 'apis', 'beginner', 'updated', 'apis', 'beginner', 'updated', 'apis', 'beginner', 'updated']
• spotify.com is it opens up at this api.
  → ['spotify', 'com', 'open', 'api']

Stopwords removed (exact count on originals): total=69519 (comments=9210, captions=60309)

Top tokens — Comments (10)
[('http', 293), ('href', 289), ('watch', 289), ('youtube', 283), ('com', 275), ('amp', 274), ('www', 272), ('wxsd', 268), ('zgxjrw', 268), ('api', 2

In [10]:
# Save cleaned data
comments_df.to_csv('cleaned_comments.csv', index=False)
captions_df.to_csv('cleaned_captions.csv', index=False)